In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression

# 気候データのみからやる

In [2]:
#tsvファイルの読み込み
train=pd.read_csv("./train.tsv",delimiter="\t")
weather=pd.read_csv("./weather.tsv",delimiter="\t")
test=pd.read_csv("./test.tsv",delimiter="\t")

#年月日の書き方がバラバラなので揃える
train["datetime"]=pd.DatetimeIndex(train["datetime"].values)
test["datetime"]=pd.DatetimeIndex(test["datetime"].values)
weather["年月日"]=pd.DatetimeIndex(weather["年月日"].apply(lambda x:x.replace("/","-")).values)


In [3]:
train.head()

,datetime,park,visitors
0,2015-01-01,阿寒摩周国立公園,11028
1,2015-01-01,十和田八幡平国立公園,34757
2,2015-01-01,日光国立公園,29714
3,2015-01-01,伊勢志摩国立公園,42652
4,2015-01-01,大山隠岐国立公園,3637


In [4]:
#気候情報・地点・公園を全て揃える
weather_useful_cols=[c for c in weather.columns.difference(["年月日","地点","最多風向(16方位)"]) if weather[c].dropna().shape[0]>=10000]
chitens=weather["地点"].drop_duplicates().values
parks=train["park"].drop_duplicates().values

In [5]:
weather_useful_cols

['10分間降水量の最大(mm)',
 '平均気温(℃)',
 '平均風速(m/s)',
 '日照時間(時間)',
 '最低気温(℃)',
 '最大瞬間風速(m/s)',
 '最大風速(m/s)',
 '最高気温(℃)',
 '降水量の合計(mm)']

In [6]:
chitens

array(['十和田', '大山', '大田', '日光', '渡嘉敷', '熊本', '釧路', '青森', '高森', '鳥羽',
       '鹿児島', '鹿角'], dtype=object)

In [7]:
parks

array(['阿寒摩周国立公園', '十和田八幡平国立公園', '日光国立公園', '伊勢志摩国立公園', '大山隠岐国立公園',
       '阿蘇くじゅう国立公園', '霧島錦江湾国立公園', '慶良間諸島国立公園'], dtype=object)

In [8]:
#[地点_気候]というカラムを追加していく
for chiten in chitens:
    df=weather[weather["地点"]==chiten]
    for col in weather_useful_cols:
        df_=df[["年月日",col]]
        df_.columns=["datetime",chiten+"_"+col]
        train=pd.merge(train,df_,on="datetime",how="left")
        test=pd.merge(test,df_,on="datetime",how="left")


In [9]:
train.head()

,datetime,park,visitors,十和田_10分間降水量の最大(mm),十和田_平均気温(℃),十和田_平均風速(m/s),十和田_日照時間(時間),十和田_最低気温(℃),十和田_最大瞬間風速(m/s),十和田_最大風速(m/s),...,鹿児島_降水量の合計(mm),鹿角_10分間降水量の最大(mm),鹿角_平均気温(℃),鹿角_平均風速(m/s),鹿角_日照時間(時間),鹿角_最低気温(℃),鹿角_最大瞬間風速(m/s),鹿角_最大風速(m/s),鹿角_最高気温(℃),鹿角_降水量の合計(mm)
0,2015-01-01,阿寒摩周国立公園,11028,1.0,-1.9,2.8,2.1,-2.9,13.0,7.5,...,3.0,0.5,-2.9,2.7,2.2,-5.6,9.5,6.7,-1.4,2.5
1,2015-01-01,十和田八幡平国立公園,34757,1.0,-1.9,2.8,2.1,-2.9,13.0,7.5,...,3.0,0.5,-2.9,2.7,2.2,-5.6,9.5,6.7,-1.4,2.5
2,2015-01-01,日光国立公園,29714,1.0,-1.9,2.8,2.1,-2.9,13.0,7.5,...,3.0,0.5,-2.9,2.7,2.2,-5.6,9.5,6.7,-1.4,2.5
3,2015-01-01,伊勢志摩国立公園,42652,1.0,-1.9,2.8,2.1,-2.9,13.0,7.5,...,3.0,0.5,-2.9,2.7,2.2,-5.6,9.5,6.7,-1.4,2.5
4,2015-01-01,大山隠岐国立公園,3637,1.0,-1.9,2.8,2.1,-2.9,13.0,7.5,...,3.0,0.5,-2.9,2.7,2.2,-5.6,9.5,6.7,-1.4,2.5


In [10]:
#公園ごとにtrain_,test_を作る。何も考えずにAdaBoostを使う。
#all_testに答えをどんどん蓄積
all_test=pd.DataFrame()
for park in parks:
    train_=train[train["park"]==park].fillna(0)
    test_=test[test["park"]==park].fillna(0).reset_index()
    x_col=train_.columns.difference(["datetime","park","visitors"])
    y_col="visitors"
    test_["visitors"]=AdaBoostRegressor().fit(train_[x_col].values,train_[y_col].values).predict(test_[x_col])
    all_test=all_test.append(test_)

In [11]:
all_test.head()

,level_0,index,datetime,park,十和田_10分間降水量の最大(mm),十和田_平均気温(℃),十和田_平均風速(m/s),十和田_日照時間(時間),十和田_最低気温(℃),十和田_最大瞬間風速(m/s),...,鹿角_10分間降水量の最大(mm),鹿角_平均気温(℃),鹿角_平均風速(m/s),鹿角_日照時間(時間),鹿角_最低気温(℃),鹿角_最大瞬間風速(m/s),鹿角_最大風速(m/s),鹿角_最高気温(℃),鹿角_降水量の合計(mm),visitors
0,0,0,2017-01-01,阿寒摩周国立公園,0.0,0.1,2.3,5.5,-7.3,9.3,...,0.5,-1.9,0.6,0.4,-7.3,5.4,2.4,2.3,0.5,5341.560976
1,8,8,2017-01-02,阿寒摩周国立公園,0.5,2.2,1.2,3.0,-1.5,6.3,...,0.5,-0.4,0.5,2.2,-5.8,3.2,2.2,3.9,5.0,7205.751678
2,16,16,2017-01-03,阿寒摩周国立公園,0.5,0.7,1.4,2.4,-0.8,5.2,...,0.5,0.0,0.8,0.9,-2.8,3.7,2.2,2.2,0.5,5341.560976
3,24,24,2017-01-04,阿寒摩周国立公園,0.0,0.3,1.5,3.8,-2.7,8.4,...,1.0,-1.2,0.7,2.9,-4.4,7.4,4.8,2.7,2.5,5464.958904
4,32,32,2017-01-05,阿寒摩周国立公園,0.5,-1.7,2.3,3.8,-4.8,10.9,...,1.0,-2.4,1.3,3.4,-6.7,8.4,4.7,0.6,2.0,4884.250000


In [12]:
#提出
all_test.sort_values(by="index")[["index","visitors"]].to_csv("test1.tsv",sep="\t",header=None,index=None)